In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# DATA

## Import data

In [84]:
final_df=pd.read_csv("D:/HUST/20241/DS/Project/prj_data/final_data.csv")
test = pd.read_csv("D:/HUST/20241/DS/Project/prj_data/nfts_predict.csv")
y_test = pd.read_csv("D:/HUST/20241/DS/Project/Result.csv")

In [85]:
final_df.head()

,Unnamed: 0,global_index,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_date,last_sale_price,...,nft_seller_fees_min,nft_seller_fees_max,nft_seller_fees_mean,nft_seller_fees_sum,nft_seller_fees_median,nft_platform_fees_min,nft_platform_fees_max,nft_platform_fees_mean,nft_platform_fees_sum,nft_platform_fees_median
0,0,21928,0,49,2.000000,0.0,0.0,0.0,2022-11-01,1.266732,...,200.0,1000.0,713.541667,34250.0,750.0,250,250,250.0,12000,250.0
1,1,32621,0,45,97.077216,0.0,0.0,0.0,2021-08-01,1.200357,...,200.0,1000.0,713.541667,34250.0,750.0,250,250,250.0,12000,250.0
2,2,32622,1,45,139.453086,0.0,0.0,0.0,2022-08-01,3.180572,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0
3,3,32623,2,45,127.753445,0.0,0.0,0.0,2021-08-01,3.502910,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0
4,4,32624,3,45,112.315719,0.0,0.0,0.0,2022-09-01,4.009567,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0


In [86]:
final_df.drop(columns=['Unnamed: 0'],inplace=True)

In [87]:
df_Train=final_df.iloc[:final_df.shape[0]-test.shape[0],:]
df_Test=final_df.iloc[final_df.shape[0]-test.shape[0]:,:]

In [88]:
(df_Train.shape,df_Test.shape)

((309910, 61), (181214, 61))

In [89]:
df_Test.drop(['last_sale_price'],axis=1,inplace=True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_456\1282937926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Test.drop(['last_sale_price'],axis=1,inplace=True)


In [90]:
df_Test.head()

,global_index,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,last_sale_date,n_tweets_in_range,avg_likes,...,nft_seller_fees_min,nft_seller_fees_max,nft_seller_fees_mean,nft_seller_fees_sum,nft_seller_fees_median,nft_platform_fees_min,nft_platform_fees_max,nft_platform_fees_mean,nft_platform_fees_sum,nft_platform_fees_median
309910,0,0,29,176.364208,0.0,0.0,0.0,2022-06-01,2,340.5,...,200.0,1000.0,713.541667,34250.0,750.0,250,250,250.0,12000,250.0
309911,1,1,29,207.209228,0.0,0.0,0.0,2022-05-01,2,340.5,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0
309912,2,2,29,157.293414,0.0,0.0,0.0,2022-10-01,2,340.5,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0
309913,3,3,29,173.372185,0.0,0.0,0.0,2022-06-01,2,340.5,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0
309914,4,4,29,190.077584,0.0,0.0,0.0,2022-06-01,2,340.5,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0


In [91]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491124 entries, 0 to 491123
Data columns (total 61 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   global_index              491124 non-null  int64  
 1   nft_id                    491124 non-null  int64  
 2   collection_id             491124 non-null  int64  
 3   rarity_score              491124 non-null  float64
 4   openrarity_score          491124 non-null  float64
 5   openrarity_rank           491124 non-null  float64
 6   openrarity_max_rank       491124 non-null  float64
 7   last_sale_date            491124 non-null  object 
 8   last_sale_price           309910 non-null  float64
 9   n_tweets_in_range         491124 non-null  int64  
 10  avg_likes                 491124 non-null  float64
 11  avg_replies               491124 non-null  float64
 12  avg_retweets              491124 non-null  float64
 13  min_likes                 491124 non-null  i

In [92]:
final_df.isna().sum()

global_index                0
nft_id                      0
collection_id               0
rarity_score                0
openrarity_score            0
                           ..
nft_platform_fees_min       0
nft_platform_fees_max       0
nft_platform_fees_mean      0
nft_platform_fees_sum       0
nft_platform_fees_median    0
Length: 61, dtype: int64

## Preprocess data

In [93]:
# Loại bỏ cột 'last_sale_price' để tạo X_train và y_train
X_train = df_Train.drop(['last_sale_price' ,'last_sale_date', 'creation_date'], axis=1)
y_train = df_Train['last_sale_price']

### Log transform

In [94]:
cat_features = [
    'verification_status', 'contract_type', 
    'openrarity_enabled', 'has_website', 'has_own_twitter', 
    'has_discord', 'has_medium'
]

In [95]:
for col in X_train.columns:
    if col not in cat_features:
        min_val = X_train[col].min()
        if min_val < 0:  # Nếu có giá trị âm
            X_train[col] = np.log1p(X_train[col] - min_val + 1)  # Dịch dữ liệu về không gian dương
        else:
            X_train[col] = np.log1p(X_train[col])  # Log bình thường

In [96]:
# Sao chép toàn bộ dữ liệu
X_full = X_train.copy()
y_full = y_train.copy()

In [97]:
X_full.head()

,global_index,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,n_tweets_in_range,avg_likes,avg_replies,...,nft_seller_fees_min,nft_seller_fees_max,nft_seller_fees_mean,nft_seller_fees_sum,nft_seller_fees_median,nft_platform_fees_min,nft_platform_fees_max,nft_platform_fees_mean,nft_platform_fees_sum,nft_platform_fees_median
0,9.995565,0.000000,3.912023,1.098612,0.0,0.0,0.0,3.465736,5.241576,2.228973,...,5.303305,6.908755,6.571641,10.441471,6.621406,5.525453,5.525453,5.525453,9.392745,5.525453
1,10.392742,0.000000,3.828641,4.585755,0.0,0.0,0.0,2.484907,4.899196,2.671009,...,5.303305,6.908755,6.571641,10.441471,6.621406,5.525453,5.525453,5.525453,9.392745,5.525453
2,10.392773,0.693147,3.828641,4.944874,0.0,0.0,0.0,2.484907,4.899196,2.671009,...,5.303305,6.908755,6.563075,10.411840,6.621406,5.525453,5.525453,5.525453,9.371694,5.525453
3,10.392803,1.098612,3.828641,4.857899,0.0,0.0,0.0,2.484907,4.899196,2.671009,...,5.303305,6.908755,6.563075,10.411840,6.621406,5.525453,5.525453,5.525453,9.371694,5.525453
4,10.392834,1.386294,3.828641,4.730178,0.0,0.0,0.0,2.484907,4.899196,2.671009,...,5.303305,6.908755,6.563075,10.411840,6.621406,5.525453,5.525453,5.525453,9.371694,5.525453


In [98]:
y_full.head()

0    1.266732
1    1.200357
2    3.180572
3    3.502910
4    4.009567
Name: last_sale_price, dtype: float64

### Scale

In [99]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)

In [100]:
X_train_scaled.head()

,global_index,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,n_tweets_in_range,avg_likes,avg_replies,...,nft_seller_fees_min,nft_seller_fees_max,nft_seller_fees_mean,nft_seller_fees_sum,nft_seller_fees_median,nft_platform_fees_min,nft_platform_fees_max,nft_platform_fees_mean,nft_platform_fees_sum,nft_platform_fees_median
0,-3.067672,-7.210928,0.790916,-2.707906,-0.587963,-0.58584,-0.591742,1.085937,1.339369,0.474642,...,-0.237109,0.120106,0.567511,0.762391,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.759963,8.881784e-16
1,-2.566807,-7.210928,0.706391,-0.496373,-0.587963,-0.58584,-0.591742,0.179635,1.151293,0.775228,...,-0.237109,0.120106,0.567511,0.762391,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.759963,8.881784e-16
2,-2.566768,-6.611944,0.706391,-0.268621,-0.587963,-0.58584,-0.591742,0.179635,1.151293,0.775228,...,-0.237109,0.120106,0.460751,0.726307,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.732396,8.881784e-16
3,-2.566729,-6.261561,0.706391,-0.323780,-0.587963,-0.58584,-0.591742,0.179635,1.151293,0.775228,...,-0.237109,0.120106,0.460751,0.726307,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.732396,8.881784e-16
4,-2.566691,-6.012960,0.706391,-0.404780,-0.587963,-0.58584,-0.591742,0.179635,1.151293,0.775228,...,-0.237109,0.120106,0.460751,0.726307,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.732396,8.881784e-16


# Modeling

## LinearRegression

In [101]:
y_train_log = np.log1p(y_train)  # log(1 + y)

In [34]:
results = {}

In [21]:
fold = KFold(n_splits=8, shuffle=True, random_state=2023)

# Lưu kết quả
fold_rmse_lr = []
fold_r2_lr = []

# Vòng lặp qua các folds
for train_index, test_index in fold.split(X_train_scaled):
    # Chia dữ liệu thành train/test
    X_train, X_test = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    y_train, y_test = y_train_log.iloc[train_index], y_train_log.iloc[test_index]
    
    # Định nghĩa và huấn luyện mô hình
    model_lr = LinearRegression()
    model_lr.fit(X_train, y_train)
    
    # Dự đoán và đánh giá
    preds = model_lr.predict(X_test)
    rmse = root_mean_squared_error(np.expm1(y_test), np.expm1(preds))  # Tính RMSE
    r2 = r2_score(np.expm1(y_test), np.expm1(preds))
    
    print(f"Fold RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")
    
    # Lưu kết quả
    fold_rmse_lr.append(rmse)
    fold_r2_lr.append(r2)

# In kết quả tổng thể
print(f"Mean RMSE across folds: {np.mean(fold_rmse_lr):.4f}")
print(f"Mean R2 Score across folds: {np.mean(fold_r2_lr):.4f}")

Fold RMSE: 11.8298, R2 Score: 0.1627
Fold RMSE: 10.6551, R2 Score: 0.1849
Fold RMSE: 12.4644, R2 Score: 0.1550
Fold RMSE: 12.5877, R2 Score: 0.1435
Fold RMSE: 12.0867, R2 Score: 0.1562
Fold RMSE: 10.8409, R2 Score: 0.1721
Fold RMSE: 12.0351, R2 Score: 0.1648
Fold RMSE: 10.5071, R2 Score: 0.1937
Mean RMSE across folds: 11.6259
Mean R2 Score across folds: 0.1666


In [35]:
results['LR'] = np.mean(fold_rmse_lr)

## DecisionTreeRegressor

In [23]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


# Lưu kết quả
fold_rmse_dt = []
fold_r2_dt = []

# Cài đặt các hyperparameters cho DecisionTreeRegressor
params = {
    # 'max_depth': None,              # Độ sâu tối đa của cây
    'min_samples_split': 5,       # Số mẫu tối thiểu để chia một nút
    'min_samples_leaf': 2,        # Số mẫu tối thiểu ở mỗi lá
    'random_state': 2             # Đảm bảo tính tái lập
}
# Vòng lặp qua các folds
for train_index, test_index in fold.split(X_train_scaled):
    # Chia dữ liệu thành train/test
    X_train, X_test = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    y_train, y_test = y_train_log.iloc[train_index], y_train_log.iloc[test_index]
    
    # Định nghĩa và huấn luyện mô hình
    model_DT = DecisionTreeRegressor(**params)
    model_DT.fit(X_train, y_train)
    
    # Dự đoán và đánh giá
    preds = model_DT.predict(X_test)
    rmse = root_mean_squared_error(np.expm1(y_test), np.expm1(preds))  # Tính RMSE
    r2 = r2_score(np.expm1(y_test), np.expm1(preds))
    
    print(f"Fold RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")
    
    # Lưu kết quả
    fold_rmse_dt.append(rmse)
    fold_r2_dt.append(r2)

# In kết quả tổng thể
print(f"Mean RMSE across folds: {np.mean(fold_rmse_dt):.4f}")
print(f"Mean R2 Score across folds: {np.mean(fold_r2_dt):.4f}")


Fold RMSE: 9.6814, R2 Score: 0.4392
Fold RMSE: 9.2795, R2 Score: 0.3818
Fold RMSE: 10.6782, R2 Score: 0.3799
Fold RMSE: 9.8980, R2 Score: 0.4704
Fold RMSE: 10.2570, R2 Score: 0.3923
Fold RMSE: 9.1415, R2 Score: 0.4113
Fold RMSE: 9.8287, R2 Score: 0.4430
Fold RMSE: 8.1076, R2 Score: 0.5199
Mean RMSE across folds: 9.6090
Mean R2 Score across folds: 0.4297


In [36]:
results['DT'] = np.mean(fold_rmse_dt)

## RF

In [25]:
# Lưu kết quả
fold_rmse_rf = []
fold_r2_rf = []

# Cài đặt các hyperparameters
params = {
    'n_estimators': 250,        # Số lượng cây trong rừng
    'max_depth': 15,            # Độ sâu tối đa của cây
    'min_samples_split': 5,     # Số lượng mẫu tối thiểu để chia nhánh
    'min_samples_leaf': 2,      # Số lượng mẫu tối thiểu tại mỗi lá
    'random_state': 3,          # Đảm bảo tính tái lập
    'n_jobs': -1                # Sử dụng tất cả các CPU cores
}

# Vòng lặp qua các folds
for train_index, test_index in fold.split(X_train_scaled):
    # Chia dữ liệu thành train/test
    X_train, X_test = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    y_train, y_test = y_train_log.iloc[train_index], y_train_log.iloc[test_index]
    
    # Định nghĩa và huấn luyện mô hình
    model_RF = RandomForestRegressor(**params)
    model_RF.fit(X_train, y_train)
    
    # Dự đoán và đánh giá
    preds = model_RF.predict(X_test)
    rmse = root_mean_squared_error(np.expm1(y_test), np.expm1(preds))  # Tính RMSE
    r2 = r2_score(np.expm1(y_test), np.expm1(preds))
    
    print(f"Fold RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")
    
    # Lưu kết quả
    fold_rmse_rf.append(rmse)
    fold_r2_rf.append(r2)

# In kết quả tổng thể
print(f"Mean RMSE across folds: {np.mean(fold_rmse_rf):.4f}")
print(f"Mean R2 Score across folds: {np.mean(fold_r2_rf):.4f}")

Fold RMSE: 8.9697, R2 Score: 0.5186
Fold RMSE: 7.6070, R2 Score: 0.5845
Fold RMSE: 9.5099, R2 Score: 0.5081
Fold RMSE: 9.9920, R2 Score: 0.4603
Fold RMSE: 9.3103, R2 Score: 0.4993
Fold RMSE: 8.1853, R2 Score: 0.5280
Fold RMSE: 9.1134, R2 Score: 0.5211
Fold RMSE: 7.1866, R2 Score: 0.6228
Mean RMSE across folds: 8.7343
Mean R2 Score across folds: 0.5304


In [37]:
results['RF'] = np.mean(fold_rmse_rf)

## XGBRegressor

In [27]:
# Lưu kết quả
fold_rmse_xgb = []
fold_r2_xgb = []

# Hyperparameters cho XGBRegressor
params = {
    'n_estimators': 300,        # Số lượng cây
    'learning_rate': 0.05,       # Tốc độ học
    'max_depth': 8,             # Độ sâu tối đa
    'subsample': 0.8,           # Tỷ lệ mẫu ngẫu nhiên
    'colsample_bytree': 0.8,    # Tỷ lệ cột mẫu ngẫu nhiên
    'random_state': 4,          # Đảm bảo tính tái lập
    'n_jobs': -1                # Sử dụng tất cả CPU cores
}

# Vòng lặp qua các folds
for train_index, test_index in fold.split(X_train_scaled):
    # Chia dữ liệu thành train/test
    X_train, X_test = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    y_train, y_test = y_train_log.iloc[train_index], y_train_log.iloc[test_index]
    
    # Định nghĩa và huấn luyện mô hình
    model_XGB = XGBRegressor(**params)
    model_XGB.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    
    # Dự đoán và đánh giá
    preds = model_XGB.predict(X_test)
    rmse = root_mean_squared_error(np.expm1(y_test), np.expm1(preds))  # Tính RMSE
    r2 = r2_score(np.expm1(y_test), np.expm1(preds))
    
    print(f"XGBoost Fold RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")
    
    # Lưu kết quả
    fold_rmse_xgb.append(rmse)
    fold_r2_xgb.append(r2)

# In kết quả tổng thể
print(f"Mean RMSE for XGBoost: {np.mean(fold_rmse_xgb):.4f}")
print(f"Mean R2 Score for XGBoost: {np.mean(fold_r2_xgb):.4f}")


XGBoost Fold RMSE: 8.9340, R2 Score: 0.5225
XGBoost Fold RMSE: 7.5941, R2 Score: 0.5859
XGBoost Fold RMSE: 9.4602, R2 Score: 0.5133
XGBoost Fold RMSE: 10.0550, R2 Score: 0.4535
XGBoost Fold RMSE: 9.3233, R2 Score: 0.4979
XGBoost Fold RMSE: 8.1066, R2 Score: 0.5370
XGBoost Fold RMSE: 9.0933, R2 Score: 0.5232
XGBoost Fold RMSE: 7.1710, R2 Score: 0.6244
Mean RMSE for XGBoost: 8.7172
Mean R2 Score for XGBoost: 0.5322


In [38]:
results['XGB'] = np.mean(fold_rmse_xgb)

## LGBMRegressor

In [29]:
# Lưu kết quả
fold_rmse_lgbm = []
fold_r2_lgbm = []

# Hyperparameters cho LGBMRegressor
params = {
    'n_estimators': 250,        # Số lượng cây
    'learning_rate': 0.1,       # Tốc độ học
    'max_depth': 5,             # Độ sâu tối đa
    'num_leaves': 31,           # Số lá tối đa
    'subsample': 0.8,           # Tỷ lệ mẫu ngẫu nhiên
    'colsample_bytree': 0.8,    # Tỷ lệ cột mẫu ngẫu nhiên
    'random_state': 5,          # Đảm bảo tính tái lập
    'n_jobs': -1                # Sử dụng tất cả CPU cores
}

# Vòng lặp qua các folds
for train_index, test_index in fold.split(X_train_scaled):
    # Chia dữ liệu thành train/test
    X_train, X_test = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    y_train, y_test = y_train_log.iloc[train_index], y_train_log.iloc[test_index]
    
    # Định nghĩa và huấn luyện mô hình
    model_LMGB = LGBMRegressor(**params)
    model_LMGB.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    
    # Dự đoán và đánh giá
    preds = model_LMGB.predict(X_test)
    rmse = root_mean_squared_error(np.expm1(y_test), np.expm1(preds))  # Tính RMSE
    r2 = r2_score(np.expm1(y_test), np.expm1(preds))
    
    print(f"LGBM Fold RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")
    
    # Lưu kết quả
    fold_rmse_lgbm.append(rmse)
    fold_r2_lgbm.append(r2)

# In kết quả tổng thể
print(f"Mean RMSE for LGBM: {np.mean(fold_rmse_lgbm):.4f}")
print(f"Mean R2 Score for LGBM: {np.mean(fold_r2_lgbm):.4f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014400 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3327
[LightGBM] [Info] Number of data points in the train set: 271171, number of used features: 50
[LightGBM] [Info] Start training from score 0.345720
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [39]:
results['LGBM'] = np.mean(fold_rmse_lgbm)

## CatBoostRegressor

In [31]:
from catboost import CatBoostRegressor

# Lưu kết quả
fold_rmse_cat = []
fold_r2_cat = []

# Hyperparameters cho CatBoostRegressor
params = {
    'iterations': 1000,        # Số vòng lặp tối đa (cây tối đa)
    'learning_rate': 0.05,      # Tốc độ học
    'depth': 14,                # Độ sâu tối đa
    'subsample': 0.8,          # Lấy mẫu 80% dữ liệu
    'random_seed': 6,          # Đảm bảo tính tái lập
    'verbose': 0,              # Tắt thông báo
    'loss_function': 'RMSE',   # Hàm mất mát
    'eval_metric': 'RMSE'      # Hàm đánh giá
}

# Vòng lặp qua các folds
for train_index, test_index in fold.split(X_train_scaled):
    # Chia dữ liệu thành train/test
    X_train, X_test = X_train_scaled.iloc[train_index], X_train_scaled.iloc[test_index]
    y_train, y_test = y_train_log.iloc[train_index], y_train_log.iloc[test_index]
    
    # Định nghĩa và huấn luyện mô hình
    model_CAT = CatBoostRegressor(**params)
    model_CAT.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Tập đánh giá
        early_stopping_rounds=50      # Dừng sớm sau 50 vòng không cải thiện
    )
    
    # Dự đoán và đánh giá
    preds = model_CAT.predict(X_test)
    rmse = root_mean_squared_error(np.expm1(y_test), np.expm1(preds))  # Tính RMSE
    r2 = r2_score(np.expm1(y_test), np.expm1(preds))
    
    print(f"CatBoost Fold RMSE: {rmse:.4f}, R2 Score: {r2:.4f}")
    
    # Lưu kết quả
    fold_rmse_cat.append(rmse)
    fold_r2_cat.append(r2)

# In kết quả tổng thể
print(f"Mean RMSE for CatBoost: {np.mean(fold_rmse_cat):.4f}")
print(f"Mean R2 Score for CatBoost: {np.mean(fold_r2_cat):.4f}")


CatBoost Fold RMSE: 9.0011, R2 Score: 0.5153
CatBoost Fold RMSE: 7.7845, R2 Score: 0.5649
CatBoost Fold RMSE: 9.5225, R2 Score: 0.5068
CatBoost Fold RMSE: 10.0210, R2 Score: 0.4572
CatBoost Fold RMSE: 9.4557, R2 Score: 0.4836
CatBoost Fold RMSE: 8.2425, R2 Score: 0.5214
CatBoost Fold RMSE: 9.2410, R2 Score: 0.5076
CatBoost Fold RMSE: 7.2804, R2 Score: 0.6129
Mean RMSE for CatBoost: 8.8186
Mean R2 Score for CatBoost: 0.5212


In [137]:
results['Cat'] = np.mean(fold_rmse_cat)

## Result

In [138]:
print(results)
min_rmse = min(results, key=results.get)
min_value = results[min_rmse]

# In kết quả
print(f"Mô hình với rmse trung bình thấp nhất là: {min_rmse}, Giá trị: {min_value}")

{'LR': 11.62585960721023, 'DT': 9.608986856229409, 'RF': 8.734275039725333, 'XGB': 8.717178722033943, 'LGBM': 8.760102449584517, 'Cat': 8.698588140552536}
Mô hình với rmse trung bình thấp nhất là: Cat, Giá trị: 8.698588140552536


=> Chúng ra sẽ sử dụng mô hình CatBoost để dự đoán

Train lại model với toàn bộ data

In [20]:
params_cat = {
    'iterations': 1000,        # Số vòng lặp tối đa (cây tối đa)
    'learning_rate': 0.05,      # Tốc độ học
    'depth': 14,                # Độ sâu tối đa
    'subsample': 0.8,          # Lấy mẫu 80% dữ liệu
    'random_seed': 6,          # Đảm bảo tính tái lập
    'verbose': 0,              # Tắt thông báo
    'loss_function': 'RMSE',   # Hàm mất mát
    'eval_metric': 'RMSE'      # Hàm đánh giá
}

In [21]:
from catboost import CatBoostRegressor

model_CAT_Final = CatBoostRegressor(**params_cat)
model_CAT_Final.fit(X_train_scaled, y_train_log)

### Save model

In [276]:
print("Model trained successfully!")

Model trained successfully!


In [ ]:
import joblib
joblib.dump(model_CAT_Final, "cat_models_final.joblib")

### Dự đoán cho giá trị mới(result)

In [102]:
import joblib
importedModelCat = joblib.load("cat_models_final.joblib")

In [103]:
y_test.head()

,global_index,last_sale_price
0,0,4.166827
1,1,3.605910
2,2,14.558263
3,3,3.230520
4,4,3.916119


In [104]:
X_test = df_Test.drop(['last_sale_date', 'creation_date'], axis=1)   

In [105]:
X_test.head()

,global_index,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,n_tweets_in_range,avg_likes,avg_replies,...,nft_seller_fees_min,nft_seller_fees_max,nft_seller_fees_mean,nft_seller_fees_sum,nft_seller_fees_median,nft_platform_fees_min,nft_platform_fees_max,nft_platform_fees_mean,nft_platform_fees_sum,nft_platform_fees_median
309910,0,0,29,176.364208,0.0,0.0,0.0,2,340.5,68.5,...,200.0,1000.0,713.541667,34250.0,750.0,250,250,250.0,12000,250.0
309911,1,1,29,207.209228,0.0,0.0,0.0,2,340.5,68.5,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0
309912,2,2,29,157.293414,0.0,0.0,0.0,2,340.5,68.5,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0
309913,3,3,29,173.372185,0.0,0.0,0.0,2,340.5,68.5,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0
309914,4,4,29,190.077584,0.0,0.0,0.0,2,340.5,68.5,...,200.0,1000.0,707.446809,33250.0,750.0,250,250,250.0,11750,250.0


In [106]:
for col in X_test.columns:
    if col not in cat_features:
        min_val = X_test[col].min()
        if min_val < 0:  # Nếu có giá trị âm
            X_test[col] = np.log1p(X_test[col] - min_val + 1)  # Dịch dữ liệu về không gian dương
        else:
            X_test[col] = np.log1p(X_test[col])  # Log bình thường

In [107]:
X_test.head()

,global_index,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,n_tweets_in_range,avg_likes,avg_replies,...,nft_seller_fees_min,nft_seller_fees_max,nft_seller_fees_mean,nft_seller_fees_sum,nft_seller_fees_median,nft_platform_fees_min,nft_platform_fees_max,nft_platform_fees_mean,nft_platform_fees_sum,nft_platform_fees_median
309910,0.000000,0.000000,3.401197,5.178205,0.0,0.0,0.0,1.098612,5.833348,4.241327,...,5.303305,6.908755,6.571641,10.441471,6.621406,5.525453,5.525453,5.525453,9.392745,5.525453
309911,0.693147,0.693147,3.401197,5.338543,0.0,0.0,0.0,1.098612,5.833348,4.241327,...,5.303305,6.908755,6.563075,10.411840,6.621406,5.525453,5.525453,5.525453,9.371694,5.525453
309912,1.098612,1.098612,3.401197,5.064450,0.0,0.0,0.0,1.098612,5.833348,4.241327,...,5.303305,6.908755,6.563075,10.411840,6.621406,5.525453,5.525453,5.525453,9.371694,5.525453
309913,1.386294,1.386294,3.401197,5.161192,0.0,0.0,0.0,1.098612,5.833348,4.241327,...,5.303305,6.908755,6.563075,10.411840,6.621406,5.525453,5.525453,5.525453,9.371694,5.525453
309914,1.609438,1.609438,3.401197,5.252680,0.0,0.0,0.0,1.098612,5.833348,4.241327,...,5.303305,6.908755,6.563075,10.411840,6.621406,5.525453,5.525453,5.525453,9.371694,5.525453


In [108]:
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

In [109]:
X_test_scaled.head()

,global_index,nft_id,collection_id,rarity_score,openrarity_score,openrarity_rank,openrarity_max_rank,n_tweets_in_range,avg_likes,avg_replies,...,nft_seller_fees_min,nft_seller_fees_max,nft_seller_fees_mean,nft_seller_fees_sum,nft_seller_fees_median,nft_platform_fees_min,nft_platform_fees_max,nft_platform_fees_mean,nft_platform_fees_sum,nft_platform_fees_median
309910,-15.672718,-7.210928,0.273086,-0.120643,-0.587963,-0.58584,-0.591742,-1.101324,1.664441,1.843053,...,-0.237109,0.120106,0.567511,0.762391,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.759963,8.881784e-16
309911,-14.798615,-6.611944,0.273086,-0.018957,-0.587963,-0.58584,-0.591742,-1.101324,1.664441,1.843053,...,-0.237109,0.120106,0.460751,0.726307,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.732396,8.881784e-16
309912,-14.287297,-6.261561,0.273086,-0.192786,-0.587963,-0.58584,-0.591742,-1.101324,1.664441,1.843053,...,-0.237109,0.120106,0.460751,0.726307,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.732396,8.881784e-16
309913,-13.924512,-6.012960,0.273086,-0.131433,-0.587963,-0.58584,-0.591742,-1.101324,1.664441,1.843053,...,-0.237109,0.120106,0.460751,0.726307,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.732396,8.881784e-16
309914,-13.643114,-5.820130,0.273086,-0.073412,-0.587963,-0.58584,-0.591742,-1.101324,1.664441,1.843053,...,-0.237109,0.120106,0.460751,0.726307,0.340629,8.881784e-16,8.881784e-16,8.881784e-16,0.732396,8.881784e-16


In [110]:
y_pred = importedModelCat.predict(X_test)

In [111]:
y_test_price = y_test['last_sale_price']

In [112]:
y_test_price

0          4.166827
1          3.605910
2         14.558263
3          3.230520
4          3.916119
            ...    
181209     0.695341
181210     1.043323
181211     1.087914
181212     1.067761
181213     0.460563
Name: last_sale_price, Length: 181214, dtype: float64

In [113]:
y_pred = np.expm1(y_pred)

In [114]:
root_mean_squared_error(y_test_price, y_pred)

4.816175533381146